In [1]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import keras
import numpy as np

In [2]:
from pygame import mixer
mixer.init()
sound = mixer.Sound('Drowsiness detection/alarm.wav')

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# To make the red rectangle around the screen grow larger in size as the computer detects you are sleepier  ,we define a thickness variable.
thick = 2
score = 0  # The timer until which the alarm will ring.
lbl=['Close','Open']
rpred=[99]
lpred=[99]

model = load_model('Drowsiness detection/models/cnnCat2.h5')

leye_classifier = cv2.CascadeClassifier('Drowsiness detection/haar cascade files/haarcascade_lefteye_2splits.xml')
reye_classifier = cv2.CascadeClassifier('Drowsiness detection/haar cascade files/haarcascade_righteye_2splits.xml')
face_classifier = cv2.CascadeClassifier('Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml')

In [8]:
cap = cv2.VideoCapture(0)
while True:
    ret ,frame = cap.read()
    height ,width = frame.shape[:2]
    
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray , minNeighbors = 5 , scaleFactor = 1.1 , minSize = (25,25))
    left_eye = leye_classifier.detectMultiScale(gray)
    right_eye = reye_classifier.detectMultiScale(gray)
    
#     cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
    
    for x,y,w,h in faces:
        cv2.rectangle(frame ,(x,y) ,(x+w,y+h) ,(0,0,0) , 1)
        
    for x,y,w,h in left_eye:
        l_eye = frame[y:y+h ,x:x+w]
        l_eye = cv2.cvtColor(l_eye , cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye ,(24,24))  # The model was trained in this size.
        l_eye = l_eye/255
        l_eye = np.reshape(l_eye ,(24,24,1))
        l_eye = np.expand_dims(l_eye ,axis=0)
        l_pred = model.predict_classes(l_eye)
        
        if (l_pred[0] == 1):
            lbl = 'Open'
        if (l_pred[0] == 0):
            lbl = 'Closed'
        break
        
        
        
    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h , x:x+w]
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,1)
        r_eye = np.expand_dims(r_eye,axis=0)
        r_pred = model.predict_classes(r_eye)
        if(r_pred[0]==1):
            lbl='Open'
        if(r_pred[0]==0):
            lbl='Closed'   
        break
        
    if (l_pred ==0 and r_pred ==0):
        score = score + 1 
        cv2.putText(frame ,"Closed" ,(10,height-20) ,cv2.FONT_HERSHEY_SIMPLEX ,1 ,(0,0,255) ,1 ,cv2.LINE_AA)
    else:
        score = score -1
        cv2.putText(frame ,"Open" ,(10,height-20) ,cv2.FONT_HERSHEY_SIMPLEX ,1 ,(0,0,255) ,1 ,cv2.LINE_AA)
        sound.stop()
        
    if (score<0):
        score = 0
    cv2.putText(frame,'Score:'+str(score),(100,height-20), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),1,cv2.LINE_AA)
        
    if (score>15):
        sound.play()
        
        
        if (thick<16):
            thick = thick+2
        else:
            thick = thick-2
            if (thick<2):
                thick = 2
            
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thick)  # A rectangle around the whole frame.
        
    cv2.imshow('Drowsiness_Detector' , frame)
    
    if cv2.waitKey(10)==27:
        cap.release()
        cv2.destroyAllWindows()
        break


In [15]:
np.expand_dims(np.ones((24,24)).reshape((24,24,1)) , axis=0).shape

(1, 24, 24, 1)

In [27]:
np.ones((24,24)).shape[:2]

(24, 24)